In [1]:
from IPython.core.display import display, HTML, Markdown
display(HTML("<style>.container { width:98% !important; }</style>"))

In [2]:
%matplotlib inline

In [3]:
import re
import os
import sys
import json
import collections
import geoip2.database
import geoip2.errors
from cachetools import cached
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tldextract
import requests
import netaddr
import datetime
from email_security_providers import *

In [4]:
start_time = datetime.datetime.now()

In [5]:
gcp_ips = requests.get('http://www.gstatic.com/ipranges/cloud.json').json()
gcp_ips = [(rec['ipv4Prefix'], '{} ({})'.format(rec['service'], rec['scope'])) for rec in gcp_ips['prefixes'] if 'ipv4Prefix' in rec]
gcp_ips = dict(gcp_ips)
gcp_ips = netaddr.IPSet(gcp_ips.keys())

In [6]:
aws_ips = requests.get('https://ip-ranges.amazonaws.com/ip-ranges.json').json()
aws_ips = [(rec['ip_prefix'], '{} ({})'.format(rec['service'], rec['region'])) for rec in aws_ips['prefixes'] if 'ip_prefix' in rec]
aws_ips = dict(aws_ips)
aws_ips = netaddr.IPSet(aws_ips.keys())

In [7]:
# https://www.microsoft.com/en-us/download/confirmation.aspx?id=56519
azure_ips = json.load(open('data/ServiceTags_Public_20200601.json'))
azure_ips = [(rec['properties']['addressPrefixes'], '{}'.format(rec['properties']['platform'])) for rec in azure_ips['values']]
azure_ips = [(ip, note) for ips, note in azure_ips for ip in ips]
azure_ips = dict(azure_ips)
azure_ips = netaddr.IPSet(azure_ips.keys())

In [8]:
@cached(cache={})
def is_azure(ip):
    return ip is not None and ip in azure_ips

@cached(cache={})
def is_aws(ip):
    return ip is not None and ip in aws_ips

@cached(cache={})
def is_gcp(ip):
    return ip is not None and ip in gcp_ips

In [9]:
PATTERN = re.compile(r'''
    ^(?P<domain>\S+)\s
    MX\s
    (?P<preference>\d+)\s
    (?P<mailserver>\S+)\s
    (?P<adns_status>\S+)\s
    (?P<adns_code>\d+)\s
    (?P<adns_reason>\S+)\s
    "(?P<fail_message>[^"]+)"\s
    (\(\s*(?P<ip_resolutions>.*?)\s*\)|\?)''',
    re.VERBOSE|re.IGNORECASE
)

NS_PATTERN = re.compile(r'''
    ^(?P<domain>\S+)\s
    NS\s
    (?P<nameserver>\S+)\s
    (?P<adns_status>\S+)\s
    (?P<adns_code>\d+)\s
    (?P<adns_reason>\S+)\s
    "(?P<fail_message>[^"]+)"\s
    (\(\s*(?P<ip_resolutions>.*?)\s*\)|\?)''',
    re.VERBOSE|re.IGNORECASE
)

maxmind_asn = geoip2.database.Reader('GeoLite2-ASN_20200616/GeoLite2-ASN.mmdb')
maxmind_city = geoip2.database.Reader('GeoLite2-City_20200616/GeoLite2-City.mmdb')

alexa = dict([(domain, int(rank)) for (rank, domain) in [line.strip().split(',', 1) for line in open('top-1m.csv')]])

@cached(cache={})
def maxmind(ip):
    result = {'ip': ip}

    try:
        record = maxmind_asn.asn(ip)
        result['asn'] = record.autonomous_system_number
        result['asname'] = record.autonomous_system_organization
    except geoip2.errors.AddressNotFoundError:
        result['asn'] =  None
        result['asname'] = None

    try: 
        record = maxmind_city.city(ip)
        result['cc'] = record.country.iso_code
        result['country'] = record.country.name
        result['city'] = record.city.name
    except geoip2.errors.AddressNotFoundError:
        result['cc'] = None
        result['country'] = None
        result['city'] = None
    
    return result


def parse_line(line):
    mat = PATTERN.match(line)
    if mat:
        record = mat.groupdict()
        record['domain'] = record['domain'].lower()
        record['mailserver'] = record['mailserver'].lower()
        
        if 'ip_resolutions' in record and record.get('ip_resolutions'):
            record['ip_resolutions'] = re.findall(r'(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})', record['ip_resolutions'])
        if record['ip_resolutions'] is None:
            record['ip_resolutions'] = []
        return record
    else:
        return None

def parse_NS_line(line):
    mat = NS_PATTERN.match(line)
    if mat:
        record = mat.groupdict()
        record['domain'] = record['domain'].lower()
        record['nameserver'] = record['nameserver'].lower()
        
        if 'ip_resolutions' in record and record.get('ip_resolutions'):
            record['ip_resolutions'] = re.findall(r'(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})', record['ip_resolutions'])
        if record['ip_resolutions'] is None:
            record['ip_resolutions'] = []
        return record
    else:
        return None

def tldextract_enrich(host):
    if host:
        tldinfo = tldextract.extract(host)
        return tldinfo.registered_domain, tldinfo.suffix
    else:
        return '', ''

def enrich_record(record):
    record['maxmind'] = [maxmind(ip) for ip in record['ip_resolutions']]
    record['mailserver_registered_domain'], record['mailserver_suffix'] = tldextract_enrich(record['mailserver'])

    record['alexa_domain'] = alexa.get(record['domain'])
    record['alexa_mailserver_registered_domain'] = alexa.get(record['mailserver_registered_domain'])
    return record

def bulk_dns(hosts, file_name, rrtype='a', ignore_cache=False):
    if rrtype == 'a':
        regex = re.compile(r'^(?P<host>\S+)\s(?P<rrtype>\S+)\s(?P<ip>\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})')
        table_key = 'host'
        table_val = 'ip'
        key_transform = lambda x:x
        input_transform = lambda x:x
    elif rrtype == 'ptr':
        regex = re.compile(r'^(?P<rev_ip>\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})\.in-addr\.arpa\s(?P<rrtype>\S+)\s(?P<host>\S+)')
        table_key = 'rev_ip'
        table_val = 'host'
        key_transform = lambda rev_ip: '.'.join(reversed(rev_ip.split('.')))
        input_transform = lambda ip: ('.'.join(reversed(ip.split('.'))))+'.in-addr.arpa'
    elif rrtype == 'soa':
        # 128.211.130.in-addr.arpa SOA ns-gce-public1.googledomains.com cloud-dns-hostmaster@google.com 1 21600 3600 259200 300
        regex = re.compile(r'^(?P<rev_ip>\d{1,3}\.\d{1,3}\.\d{1,3})\.in-addr\.arpa\s(?P<rrtype>\S+)\s(?P<host_hostmaster>\S+\s\S+)')
        table_key = 'rev_ip'
        table_val = 'host_hostmaster'
        key_transform = lambda rev_ip: '.'.join(reversed(rev_ip.split('.')))
        input_transform = lambda ip: ('.'.join(reversed(ip.split('.')[:3])))+'.in-addr.arpa.'
    else:
        raise Exception(f"unsupported rrtype: {rrtype}")
    
    results_file = f'{file_name}-{rrtype}-adnshost-results.txt'
    adns_input = f'/tmp/{file_name}-{rrtype}-adnshost-input.txt'
    
    if ignore_cache or not os.path.exists(results_file):
        with open(adns_input, 'w') as o:
            print('\n'.join([input_transform(d) for d in hosts if d]), file=o)

        command = f'''
        cat {adns_input} | adnshost \
            --asynch \
            --config "nameserver 8.8.8.8" \
            --type {rrtype} \
            --pipe \
            --cname-loose \
            ----addr-ipv4-only > {results_file}
        '''
        print(re.sub(r'\s+', ' ', command))
        os.system(command)
        #os.unlink(adns_input)

    table = collections.defaultdict(set)
    for line in open(results_file):
        mat = regex.search(line.strip())
        if mat:
            d = mat.groupdict()
            table[key_transform(d[table_key])].add(d[table_val])
    return table

In [10]:
df = pd.DataFrame([enrich_record(rec) for rec in [parse_line(line) for line in open('all-popular-domains-MX-20200620.txt.unique')] if rec])

In [11]:
df['mx_ip'] = df.ip_resolutions.map(lambda l: l[0] if l else None)

In [12]:
def get_maxmind_field(mm, name):
    return mm[0].get(name) if len(mm) > 0 else None

df['maxmind_ip']      = df.maxmind.map(lambda mm: get_maxmind_field(mm, 'ip'))
df['maxmind_cc']      = df.maxmind.map(lambda mm: get_maxmind_field(mm, 'cc'))
df['maxmind_asn']     = df.maxmind.map(lambda mm: get_maxmind_field(mm, 'asn'))
df['maxmind_asname']  = df.maxmind.map(lambda mm: get_maxmind_field(mm, 'asname'))
df['maxmind_country'] = df.maxmind.map(lambda mm: get_maxmind_field(mm, 'country'))
df['maxmind_city']    = df.maxmind.map(lambda mm: get_maxmind_field(mm, 'city'))

In [13]:
df['is_azure'] = df.mx_ip.map(is_azure)
df['is_aws'] = df.mx_ip.map(is_aws)
df['is_gcp'] = df.mx_ip.map(is_gcp)

In [14]:
all_ips = set()
for idx, row in df.ip_resolutions.iteritems():
    for ip in row:
        all_ips.add(ip)
print('Found {} unqiue IPs'.format(len(all_ips)))

Found 759103 unqiue IPs


In [15]:
table = bulk_dns(all_ips, 'mx_ips', 'ptr', ignore_cache=True)
df['mx_ip_ptrs'] = df.ip_resolutions.map(lambda ips: [list(table.get(ip))[0] for ip in ips if table.get(ip)])
df['mx_ip_ptr'] = df.mx_ip_ptrs.map(lambda ptrs: ptrs[0] if ptrs else '')

 cat /tmp/mx_ips-ptr-adnshost-input.txt | adnshost --asynch --config "nameserver 8.8.8.8" --type ptr --pipe --cname-loose ----addr-ipv4-only > mx_ips-ptr-adnshost-results.txt 


In [16]:
def three_octects(ip):
    return ip[:ip.rindex('.')]

table = bulk_dns(all_ips, 'mx_ips', 'soa', ignore_cache=True)
df['mx_ip_soas'] = df.ip_resolutions.map(lambda ips: [list(table.get(three_octects(ip)))[0] for ip in ips if table.get(three_octects(ip))])

df['mx_ip_soa'] = df.mx_ip_soas.map(lambda soas: soas[0] if soas else '')
df['mx_ip_soa_nameserver'] = df.mx_ip_soa.map(lambda val: val.split(' ')[0])
df['mx_ip_soa_hostmaster'] = df.mx_ip_soa.map(lambda val: val.split(' ')[1] if len(val.split(' ')) > 1 else '')
df['mx_ip_ptr_registered_domain'], df['mx_ip_ptr_suffix'] = zip(*df.mx_ip_ptr.map(tldextract_enrich))

 cat /tmp/mx_ips-soa-adnshost-input.txt | adnshost --asynch --config "nameserver 8.8.8.8" --type soa --pipe --cname-loose ----addr-ipv4-only > mx_ips-soa-adnshost-results.txt 


In [17]:
df.head(3).T

,0,1,2
domain,clothes2order.com,famima.vn,brandofsacrifice.com
preference,10,1,5
mailserver,alt4.aspmx.l.google.com,mail.famima.vn,alt2.aspmx.l.google.com
adns_status,ok,ok,ok
adns_code,0,0,0
adns_reason,ok,ok,ok
fail_message,OK,OK,OK
ip_resolutions,[209.85.233.26],[103.252.255.41],[142.250.13.26]
maxmind,"[{'ip': '209.85.233.26', 'asn': 15169, 'asname...","[{'ip': '103.252.255.41', 'asn': 45544, 'asnam...","[{'ip': '142.250.13.26', 'asn': 15169, 'asname..."
mailserver_registered_domain,google.com,famima.vn,google.com


In [18]:
display(Markdown('## Top PTR Base Domains'))
pd.DataFrame(df.mx_ip_ptr_registered_domain.value_counts())[:30]

## Top PTR Base Domains

,mx_ip_ptr_registered_domain
,12308003
secureserver.net,309113
mailspamprotection.com,65432
deteque.com,36682
outlook.com,30406
myregisteredsite.com,24888
1e100.net,9789
amazonaws.com,4764
ocn.ad.jp,4195
messagelabs.com,4104


In [19]:
display(Markdown('## Top MX Base Domains'))
pd.DataFrame(df.mailserver_registered_domain.value_counts())[:30]

## Top MX Base Domains

,mailserver_registered_domain
google.com,4018222
googlemail.com,1245901
secureserver.net,638784
outlook.com,539918
ovh.net,256850
registrar-servers.com,246956
mailspamprotection.com,197132
one.com,119823
zoho.com,108058
emailsrvr.com,94131


In [20]:
display(Markdown('## Top MX Countries'))
pd.DataFrame(df.maxmind_country.value_counts())[:30]

## Top MX Countries

,maxmind_country
United States,8435220
Germany,719477
France,482903
United Kingdom,313638
Japan,267244
Russia,220472
Netherlands,181612
Canada,177305
Denmark,152091
Bulgaria,131683


In [21]:
display(Markdown('## Top MX Cities'))
df[['maxmind_city', 'maxmind_country']].groupby(['maxmind_city', 'maxmind_country']).size().to_frame().reset_index().sort_values(0, ascending=False)[:30]

## Top MX Cities

,maxmind_city,maxmind_country,0
511,Althornbach,Germany,107517
2309,Boardman,United States,89798
21028,Vienna,Austria,66573
2594,Boydton,United States,60932
967,Ashburn,United States,59790
606,Amsterdam,Netherlands,56521
8215,Helsinki,Finland,54116
5375,Dublin,Ireland,53313
13125,Moscow,Russia,52179
5027,Des Moines,United States,51147


In [22]:
display(Markdown('## Top MX ASNs'))
pd.DataFrame(df.maxmind_asname.value_counts())[:30]

## Top MX ASNs

,maxmind_asname
GOOGLE,5249464
AS-26496-GO-DADDY-COM-LLC,657744
MICROSOFT-CORP-MSN-AS-BLOCK,502882
OVH SAS,358379
NAMECHEAP-NET,335341
1&1 Ionos Se,323412
UNIFIEDLAYER-AS-1,199197
AMAZON-02,191057
GOOGLE-PRIVATE-CLOUD,126473
RACKSPACE,119195


In [23]:
display(Markdown('## Top MX ASN and MX IP PTRs'))
df[df.mx_ip_ptr_registered_domain != ''][['domain', 'maxmind_asname', 'mx_ip_ptr_registered_domain']].\
    groupby(['maxmind_asname', 'mx_ip_ptr_registered_domain']).\
        count().reset_index().sort_values('domain', ascending=False)[:10]

## Top MX ASN and MX IP PTRs

,maxmind_asname,mx_ip_ptr_registered_domain,domain
595,AS-26496-GO-DADDY-COM-LLC,secureserver.net,308999
3736,GOOGLE-PRIVATE-CLOUD,mailspamprotection.com,49025
2445,DETEQUE,deteque.com,36682
6634,MICROSOFT-CORP-MSN-AS-BLOCK,outlook.com,30406
6919,NETWORK-SOLUTIONS-HOSTING,myregisteredsite.com,24888
3735,GOOGLE-2,mailspamprotection.com,10944
3722,GOOGLE,1e100.net,9747
3727,GOOGLE,mailspamprotection.com,5463
7051,NTT Communications Corporation,ocn.ad.jp,4195
331,AMAZON-02,amazonaws.com,3360


In [24]:
display(Markdown('## Top MX ASN and MX registered domains'))
df[['domain', 'maxmind_asname', 'mailserver_registered_domain']].\
    groupby(['maxmind_asname', 'mailserver_registered_domain']).\
        count().reset_index().sort_values('domain', ascending=False)[:10]

## Top MX ASN and MX registered domains

,maxmind_asname,mailserver_registered_domain,domain
539670,GOOGLE,google.com,3980844
539675,GOOGLE,googlemail.com,1230764
114845,AS-26496-GO-DADDY-COM-LLC,secureserver.net,627438
933873,MICROSOFT-CORP-MSN-AS-BLOCK,outlook.com,492562
1100218,OVH SAS,ovh.net,256228
967476,NAMECHEAP-NET,registrar-servers.com,246046
551222,GOOGLE-PRIVATE-CLOUD,mailspamprotection.com,126416
1124674,One.com A/S,one.com,118895
1752401,ZOHO-AS,zoho.com,107503
1208031,RACKSPACE,emailsrvr.com,93884


In [25]:
display(Markdown('## Top MX registered domains and MX IP PTRs'))
domain_ptr = df[['domain', 'mailserver_registered_domain', 'mx_ip_ptr_registered_domain']].\
    groupby(['mailserver_registered_domain', 'mx_ip_ptr_registered_domain']).\
        count().reset_index()
domain_ptr[
    (domain_ptr.mailserver_registered_domain != domain_ptr.mx_ip_ptr_registered_domain) & 
    (domain_ptr.mx_ip_ptr_registered_domain != '')
].sort_values('domain', ascending=False)

## Top MX registered domains and MX IP PTRs

,mailserver_registered_domain,mx_ip_ptr_registered_domain,domain
1210584,netsolmail.net,myregisteredsite.com,19530
780999,hostedmxserver.com,deteque.com,16695
1090639,mb1p.com,deteque.com,9264
1047454,m2bp.com,deteque.com,9056
701234,google.com,1e100.net,8410
...,...,...,...
668860,gearbeast.com,bluehost.com,1
668885,gearforkidz.com,mivamerchant.net,1
668887,geargeekslive.com,dizinc.com,1
668945,gearspear.com,anthonyidi.com,1


In [26]:
domain_ptr[domain_ptr.mx_ip_ptr_registered_domain == 'amazonaws.com'].sort_values('domain', ascending=False)

,mailserver_registered_domain,mx_ip_ptr_registered_domain,domain
418208,daemonmail.net,amazonaws.com,1671
1056705,mailgun.org,amazonaws.com,1339
617542,flockmail.com,amazonaws.com,944
1176980,mxrecord.io,amazonaws.com,129
388191,corusent.com,amazonaws.com,96
...,...,...,...
597057,feg.org.ec,amazonaws.com,1
586196,false.dk,amazonaws.com,1
584336,failsafeisolation.com,amazonaws.com,1
579929,eyedraw.eu,amazonaws.com,1


In [27]:
domain_ptr[domain_ptr.mx_ip_ptr_registered_domain == 'amazonaws.com'].domain.sum()

4764

In [28]:
domain_ptr[domain_ptr.mx_ip_ptr_registered_domain == 'googleusercontent.com'].sort_values('domain', ascending=False)

,mailserver_registered_domain,mx_ip_ptr_registered_domain,domain
1209940,netmar.com,googleusercontent.com,9
1184831,mysecurecloudhost.com,googleusercontent.com,8
1565856,siteground.biz,googleusercontent.com,3
1573223,sli-systems.com,googleusercontent.com,2
1054708,magnt.com,googleusercontent.com,2
...,...,...,...
614659,flagstaffmarathon.com,googleusercontent.com,1
612853,fitnessequipmentslab.com,googleusercontent.com,1
608100,finduxevents.com,googleusercontent.com,1
597452,feliceatestaccio.it,googleusercontent.com,1


In [29]:
domain_ptr[domain_ptr.mx_ip_ptr_registered_domain == 'googleusercontent.com'].domain.sum()

436

In [30]:
df[df.mx_ip_soa_nameserver != ''][['domain', 'maxmind_asname', 'mx_ip_soa_nameserver']].\
    groupby(['maxmind_asname', 'mx_ip_soa_nameserver']).\
        count().reset_index().sort_values('domain', ascending=False)[:10]

,maxmind_asname,mx_ip_soa_nameserver,domain
6,Messagelabs Limited,ns-161.awsdns-20.com,7037
7,PROOFPOINT-ASN-US-WEST,ns1.proofpoint.com,475
9,UK Dedicated Servers Limited,ns-1633.awsdns-12.co.uk,48
5,GOOGLE-2,ns-gce-public1.googledomains.com,14
8,TEKTONIC,hspc.tektonic.net,13
2,AS-MARCHESE-NET,ns1.mcpinc.com,7
0,1&1 Ionos Se,rns.ui-dns.com,3
1,AMAZON-AES,dns-external-master.amazon.com,2
4,COGENT-174,auth1.dns.cogentco.com,2
3,ASN-CXA-ALL-CCI-22773-RDC,ns2.coxmail.com,1


In [31]:
pd.DataFrame(df.mx_ip_soa_nameserver.value_counts())[:10]

,mx_ip_soa_nameserver
,12902948
ns-161.awsdns-20.com,7037
ns1.proofpoint.com,475
ns-1633.awsdns-12.co.uk,48
ns-gce-public1.googledomains.com,14
hspc.tektonic.net,13
ns1.mcpinc.com,7
rns.ui-dns.com,3
auth1.dns.cogentco.com,2
dns-external-master.amazon.com,2


In [32]:
pd.DataFrame(df.mx_ip_soa_hostmaster.value_counts())[:10]

,mx_ip_soa_hostmaster
,12902948
awsdns-hostmaster@amazon.com,7085
ops@proofpoint.com,475
cloud-dns-hostmaster@google.com,14
root@hspc.tektonic.net,13
webmaster@mcpinc.com,7
dnsadmin@ionos.com,3
root@amazon.com,2
dns@cogentco.com,2
dnsadmin@coxmail.com,1


In [33]:
display(Markdown('### MX Domains whose IPs are hosted in Azure'))
pd.DataFrame(df[df.is_azure].groupby(['mailserver_registered_domain']).size()).reset_index().sort_values(0, ascending=False)[:20]

### MX Domains whose IPs are hosted in Azure

,mailserver_registered_domain,0
743,icoremail.net,364
1350,scanscope.net,310
955,mailinblack.com,259
236,cali.co.uk,141
209,brightberri.net,141
454,dotmailer.co.uk,130
997,menufy.com,120
467,dsmail.es,91
694,helionmail.com,89
147,azure.com,44


In [34]:
display(Markdown('### MX Domains whose IPs are hosted in AWS'))
pd.DataFrame(df[df.is_aws].groupby(['mailserver_registered_domain']).size()).reset_index().sort_values(0, ascending=False)[:20]

### MX Domains whose IPs are hosted in AWS

,mailserver_registered_domain,0
8300,h-email.net,27200
11971,mailgun.org,25229
15314,pickelhost.com,17665
12569,messagelabs.com,13035
960,amazonaws.com,11657
1902,b-io.co,6448
2051,barracudanetworks.com,4394
18116,sophos.com,4197
20226,trendmicro.com,4136
11970,mailguard.com.au,4106


In [35]:
display(Markdown('### MX Domains whose IPs are hosted in GCP'))
pd.DataFrame(df[df.is_gcp].groupby(['mailserver_registered_domain']).size()).reset_index().sort_values(0, ascending=False)[:20]

### MX Domains whose IPs are hosted in GCP

,mailserver_registered_domain,0
6239,mailspamprotection.com,62642
7132,neen.it,171
5261,jouwweb.nl,143
1828,ccnotifier.nl,59
9228,siteground.biz,44
4004,getontheweb.com,40
10923,uservers.net,37
9053,sgvps.net,33
1696,capnova.com,24
6234,mailcannon.net,18


In [37]:
def email_provider_normalized(tup):
    for item in tup:
        if item != '':
            return item
    return ''

df['email_provider_from_asnname'] = df.maxmind_asname.map(email_provider_asns).replace(np.nan, '')
df['email_provider_from_mailserver'] = df.mailserver_registered_domain.map(email_provider_domains).replace(np.nan, '')
df['email_provider_from_ptr'] = df.mx_ip_ptr_registered_domain.map(email_provider_domains).replace(np.nan, '')
df['email_provider'] = pd.Series(zip(df.email_provider_from_mailserver, df.email_provider_from_asnname, df.email_provider_from_ptr)).map(email_provider_normalized)

In [38]:
def domain_count_by_key(p_df, keyname='email_provider'):
    '''
    Conceptually similar to this SQL query:
        SELECT 
            keyname, 
            COUNT(DISTINCT(domain)) as count
        FROM 
            p_df
        WHERE
            keyname!=''
        GROUP BY
            keyname
        ORDER BY
            count DESC
    '''
    return p_df[p_df[keyname] != ''][[keyname, 'domain']].\
            drop_duplicates().\
            groupby([keyname]).\
            size().\
            sort_values(0, ascending=False).\
            to_frame().\
            reset_index().\
            rename(columns={0: 'count'})

In [39]:
display(Markdown('### Top Email Security Providers'))
domain_count_by_key(df)

### Top Email Security Providers

,email_provider,count
0,Proofpoint,42310
1,Mimecast,36064
2,Deteque,34759
3,Barracuda,22897
4,Symantec,13832
5,Solarwinds,13516
6,AppRiver,11149
7,Protonmail,8158
8,Trend Micro,8054
9,Cisco Ironport,7504


In [40]:
display(Markdown('### Top Email Providers (as determined by ASN Name)'))
domain_count_by_key(df, 'email_provider_from_asnname')

### Top Email Providers (as determined by ASN Name)

,email_provider_from_asnname,count
0,Proofpoint,40571
1,Mimecast,35385
2,Deteque,34759
3,AppRiver,10081
4,Symantec,7320
5,Cisco Ironport,6982
6,Symantec,6887
7,Protonmail,4557
8,Forcepoint,3680
9,hornetsecurity,2009


In [41]:
display(Markdown('### Top Email Security Providers (as determined by MX Domain)'))
domain_count_by_key(df, 'email_provider_from_mailserver')

### Top Email Security Providers (as determined by MX Domain)

,email_provider_from_mailserver,count
0,Proofpoint,41735
1,Mimecast,36055
2,Barracuda,22885
3,Symantec,13813
4,Solarwinds,11973
5,AppRiver,11143
6,Protonmail,8156
7,Cisco Ironport,7124
8,Trend Micro,7047
9,Forcepoint,3924


In [42]:
display(Markdown('### Top Email Security Providers (as determined by PTR Domain)'))
domain_count_by_key(df, 'email_provider_from_ptr')

### Top Email Security Providers (as determined by PTR Domain)

,email_provider_from_ptr,count
0,Deteque,27047
1,Symantec,4032
2,Solarwinds,1599
3,Proofpoint,1300
4,SecureMX,1213
5,AppRiver,948
6,Forcepoint,880
7,Panda Security,334
8,FireEye,223
9,MX Relay,150


In [43]:
display(Markdown('### Top Email Security Providers Hosted in AWS'))
domain_count_by_key(df[df.is_aws])

### Top Email Security Providers Hosted in AWS

,email_provider,count
0,Symantec,12864
1,Trend Micro,6493
2,Barracuda,2253
3,Sophos,2114
4,vadesecure,1583
5,Mailprotector,1234
6,FireEye,963
7,DuoCircle,929
8,SpamTitan,662
9,AVG,393


In [44]:
display(Markdown('### Top Email Providers Hosted in Azure'))
domain_count_by_key(df[df.is_azure])

### Top Email Providers Hosted in Azure

,email_provider,count
0,Mail in Black,244
1,Censornet,200


In [45]:
display(Markdown('### Top Email Providers Hosted in GCP'))
domain_count_by_key(df[df.is_gcp | df.maxmind_asname.str.contains('GOOGLE')])

### Top Email Providers Hosted in GCP

,email_provider,count


In [46]:
display(Markdown('### Top Email Security Providers (self-hosted)'))
domain_count_by_key(df[df.email_provider_from_asnname != ''])

### Top Email Security Providers (self-hosted)

,email_provider,count
0,Proofpoint,40571
1,Mimecast,35385
2,Deteque,34759
3,Symantec,13523
4,AppRiver,10081
5,Cisco Ironport,6982
6,Protonmail,4557
7,Forcepoint,3680
8,hornetsecurity,2009
9,Trend Micro,1411


In [47]:
display(Markdown('### Top Email Security Providers hosted in Linode'))
domain_count_by_key(df[df.maxmind_asname == 'Linode, LLC'])

### Top Email Security Providers hosted in Linode

,email_provider,count


In [48]:
display(Markdown('### Top Email Security Providers hosted in Digital Ocean'))
domain_count_by_key(df[df.maxmind_asname == 'DIGITALOCEAN-ASN'])

### Top Email Security Providers hosted in Digital Ocean

,email_provider,count


In [49]:
display(Markdown('### Top Email Security Providers hosted in Rackspace'))
domain_count_by_key(df[df.maxmind_asname == 'RACKSPACE'])

### Top Email Security Providers hosted in Rackspace

,email_provider,count


In [50]:
display(Markdown('### Top Non-self hosted ASNs of Email Security Providers'))
df[(df.email_provider != '') & (~df.maxmind_asname.isin(email_provider_asns.keys()))].maxmind_asname.value_counts().to_frame().reset_index()[:30]

### Top Non-self hosted ASNs of Email Security Providers

,index,maxmind_asname
0,AMAZON-02,68414
1,AMAZON-AES,25547
2,COGENT-174,14827
3,Hostway Deutschland GmbH,6347
4,ZC38-AS1,4855
5,DATABANK-SLC,4854
6,j2 Global Ireland Limited,3447
7,CENTURYLINK-LEGACY-SAVVIS,3446
8,MK Netzdienste GmbH & Co. KG,3187
9,USINTERNET,3153


In [51]:
display(Markdown('### Top unlabeled MX registered domains'))
df[(~df.mailserver_registered_domain.isin(email_provider_domains.keys()))].mailserver_registered_domain.value_counts().to_frame().reset_index()[:30]

### Top unlabeled MX registered domains

,index,mailserver_registered_domain
0,google.com,4018222
1,googlemail.com,1245901
2,secureserver.net,638784
3,outlook.com,539918
4,ovh.net,256850
5,registrar-servers.com,246956
6,mailspamprotection.com,197132
7,one.com,119823
8,zoho.com,108058
9,emailsrvr.com,94131


In [1]:
f1000 = [domain.strip() for domain in open('data/f1000-domains.txt')]
f100 = f1000[:100]
f50 = f1000[:100]
f10 = f1000[:10]

In [53]:
pd.options.display.max_rows = 100
display(Markdown('### Fortune 1000 Email Security Providers'))
domain_count_by_key(df[df.domain.isin(f1000)])

### Fortune 1000 Email Security Providers

,email_provider,count
0,Proofpoint,340
1,Cisco Ironport,75
2,Mimecast,65
3,Symantec,54
4,FireEye,14
5,Trend Micro,4
6,Forcepoint,4
7,Barracuda,4
8,Postini,2
9,Fortinet,1


In [54]:
display(Markdown('### Fortune 100 Email Security Providers'))
domain_count_by_key(df[df.domain.isin(f100)])

### Fortune 100 Email Security Providers

,email_provider,count
0,Proofpoint,35
1,Cisco Ironport,8
2,Symantec,5
3,Mimecast,1
4,FireEye,1


In [55]:
display(Markdown('### Fortune 50 Email Security Providers'))
domain_count_by_key(df[df.domain.isin(f50)])

### Fortune 50 Email Security Providers

,email_provider,count
0,Proofpoint,35
1,Cisco Ironport,8
2,Symantec,5
3,Mimecast,1
4,FireEye,1


In [56]:
display(Markdown('### Fortune 10 Email Security Providers'))
domain_count_by_key(df[df.domain.isin(f10)])

### Fortune 10 Email Security Providers

,email_provider,count
0,Proofpoint,3
1,Symantec,1


In [57]:
display(Markdown('### Fortune 10 Summary'))
df[df.domain.isin(f10)][['domain', 'mailserver_registered_domain', 'email_provider']].drop_duplicates().sort_values('email_provider')

### Fortune 10 Summary

,domain,mailserver_registered_domain,email_provider
895915,gm.com,gm.com,
1003703,apple.com,apple.com,
1019948,berkshirehathaway.com,outlook.com,
3214516,exxonmobil.com,exxonmobil.com,
5247656,chevron.com,chevron.com,
283064,walmart.com,pphosted.com,Proofpoint
364491,ge.com,pphosted.com,Proofpoint
2980609,cvshealth.com,pphosted.com,Proofpoint
6274,ford.com,messagelabs.com,Symantec


In [58]:
display(Markdown('### Top Mailserver 2LDs'))
domain_count_by_key(df, 'mailserver_registered_domain')[:20]

### Top Mailserver 2LDs

,mailserver_registered_domain,count
0,google.com,1031219
1,outlook.com,530855
2,googlemail.com,511455
3,secureserver.net,320525
4,ovh.net,102265
5,mailspamprotection.com,65745
6,yandex.net,51512
7,emailsrvr.com,47438
8,registrar-servers.com,46698
9,dreamhost.com,45791


In [59]:
display(Markdown('### Top Mailserver 2LDs hosted in AWS'))
domain_count_by_key(df[df.is_aws], 'mailserver_registered_domain')[:20]

### Top Mailserver 2LDs hosted in AWS

,mailserver_registered_domain,count
0,h-email.net,27184
1,pickelhost.com,17653
2,messagelabs.com,12864
3,mailgun.org,12763
4,amazonaws.com,11430
5,b-io.co,6437
6,trendmicro.com,3970
7,trendmicro.eu,2525
8,mailerhost.net,2522
9,barracudanetworks.com,2204


In [60]:
display(Markdown('### Top Mailserver 2LDs hosted in Azure'))
domain_count_by_key(df[df.is_azure], 'mailserver_registered_domain')[:20]

### Top Mailserver 2LDs hosted in Azure

,mailserver_registered_domain,count
0,icoremail.net,355
1,mailinblack.com,244
2,scanscope.net,200
3,cali.co.uk,141
4,menufy.com,120
5,dotmailer.co.uk,66
6,helionmail.com,64
7,azure.com,42
8,brightberri.net,37
9,dsmail.es,32


In [61]:
display(Markdown('### Top Mailserver 2LDs hosted in GCP'))
domain_count_by_key(df[df.is_gcp], 'mailserver_registered_domain')[:20]

### Top Mailserver 2LDs hosted in GCP

,mailserver_registered_domain,count
0,mailspamprotection.com,44678
1,jouwweb.nl,143
2,neen.it,80
3,ccnotifier.nl,59
4,getontheweb.com,40
5,uservers.net,37
6,siteground.biz,32
7,sgvps.net,30
8,capnova.com,24
9,mailcannon.net,18


In [62]:
pd.DataFrame(df.mailserver_registered_domain.unique(), columns=['mailserver_registered_domain']).to_csv('mailserver_registered_domain.csv', index=False)

# Enrich with NS records (of the MX registered domains)

In [63]:
mailserver_ns = [parse_NS_line(line) for line in open('mailserver_registered_domain-NS-20200620.txt')]
mailserver_ns = pd.DataFrame(mailserver_ns).rename(columns={'domain': 'mailserver_registered_domain', 'nameserver': 'mailserver_registered_domain_nameserver'}).fillna('')
mailserver_ns = mailserver_ns[['mailserver_registered_domain','mailserver_registered_domain_nameserver']].\
    drop_duplicates().\
    groupby('mailserver_registered_domain').\
    aggregate(lambda s: ','.join(sorted(s))).reset_index()

mailserver_ns['mailserver_registered_domain_nameserver1'] = mailserver_ns.mailserver_registered_domain_nameserver.map(lambda ns: ns.split(',')[0] if ns else '')
mailserver_ns['mailserver_registered_domain_nameserver2'] = mailserver_ns.mailserver_registered_domain_nameserver.map(lambda ns: ns.split(',')[1] if len(ns.split(',')) > 1 else '')
mailserver_ns['mailserver_registered_domain_nameserver3'] = mailserver_ns.mailserver_registered_domain_nameserver.map(lambda ns: ns.split(',')[2] if len(ns.split(',')) > 2 else '')

In [64]:
df_with_ns = df.set_index('mailserver_registered_domain').join(mailserver_ns.set_index('mailserver_registered_domain')).reset_index()

In [65]:
for domainname, provider in email_provider_domains.items():
    tmp = df_with_ns[df_with_ns.mailserver_registered_domain == domainname]
    tmp = tmp[tmp.mailserver_registered_domain_nameserver1 != '']
    if len(tmp.mailserver_registered_domain_nameserver1.unique()) > 0:
        print('{} Nameservers for {} ({})'.format(len(tmp.mailserver_registered_domain_nameserver1.unique()), domainname, provider))
        print(tmp.mailserver_registered_domain_nameserver1.value_counts())
        print('---\n')

1 Nameservers for activegate-ss.jp (Activegate SS)
ns-1269.awsdns-30.org    73
Name: mailserver_registered_domain_nameserver1, dtype: int64
---

1 Nameservers for antispameurope.com (hornetsecurity)
godzilla-haj2.antispameurope.de    8201
Name: mailserver_registered_domain_nameserver1, dtype: int64
---

1 Nameservers for appriver.com (AppRiver)
hugh.ns.cloudflare.com    331
Name: mailserver_registered_domain_nameserver1, dtype: int64
---

1 Nameservers for arsmtp.com (AppRiver)
mdns1.appriver.com    21902
Name: mailserver_registered_domain_nameserver1, dtype: int64
---

1 Nameservers for avgcloud.net (AVG)
ns-1350.awsdns-40.org    789
Name: mailserver_registered_domain_nameserver1, dtype: int64
---

1 Nameservers for baesystems.com (BAE Systems)
udns1.cscdns.net    11
Name: mailserver_registered_domain_nameserver1, dtype: int64
---

1 Nameservers for barracuda.de (Barracuda)
Series([], Name: mailserver_registered_domain_nameserver1, dtype: int64)
---

1 Nameservers for barracuda.net (B

1 Nameservers for sendio.com (Sendio)
Series([], Name: mailserver_registered_domain_nameserver1, dtype: int64)
---

1 Nameservers for snwlhosted.com (Sonic Wall)
Series([], Name: mailserver_registered_domain_nameserver1, dtype: int64)
---

1 Nameservers for snwlhostedeu.com (Sonic Wall)
Series([], Name: mailserver_registered_domain_nameserver1, dtype: int64)
---

1 Nameservers for sonicwall.com (Sonic Wall)
Series([], Name: mailserver_registered_domain_nameserver1, dtype: int64)
---

1 Nameservers for sophos.com (Sophos)
a1-100.akam.net    4424
Name: mailserver_registered_domain_nameserver1, dtype: int64
---

1 Nameservers for spamexperts.com (Solarwinds)
ns-1092.awsdns-08.org    11545
Name: mailserver_registered_domain_nameserver1, dtype: int64
---

1 Nameservers for antispamcloud.com (Solarwinds)
ns-1287.awsdns-32.org    2056
Name: mailserver_registered_domain_nameserver1, dtype: int64
---

1 Nameservers for spamhero.com (SpamHero)
ns.dnsbox.net    1131
Name: mailserver_registered_do

In [66]:
def provider_search(search):
    '''
    provider a mailserver domain name or string, and get a summary of the data we have about it
    '''
    tmp = df[['domain', 'mailserver', 'mailserver_registered_domain', 'maxmind_asname']][df.mailserver.str.contains(search) | df.maxmind_asname.str.contains(search)]
    print(tmp.mailserver_registered_domain.value_counts())
    print('---')
    print(tmp.mailserver.value_counts())
    print('---')
    print(tmp.maxmind_asname.value_counts())
    print('---')
    print(tmp.domain.value_counts())

In [67]:
def explore_mailservers_by_known_asns():
    for asn, provider in email_provider_asns.items():
        tmp = df[df.maxmind_asname == asn]
        print('MXs for {} ({})'.format(asn, provider))
        print(tmp.mailserver_registered_domain.value_counts())
        print('---')

In [68]:
def explore_asns_by_known_mailservers():
    for domainname, provider in email_provider_domains.items():
        tmp = df[df.mailserver_registered_domain == domainname]
        print('ASNs for {} ({})'.format(domainname, provider))
        print(tmp.maxmind_asname.value_counts())
        print('---')

In [69]:
def identify_more_provider_asns():
    import textdistance
    asns = df.maxmind_asname.map(lambda asn: str(asn)).unique()
    for asname,provider in email_provider_asns.items():
        count = 0
        for asn in asns:
            asname_score = textdistance.jaro_winkler(asn.lower(), asname.lower())
            provider_score = textdistance.jaro_winkler(asn.lower(), provider.lower())
            if asname.lower() in asn.lower() or provider.lower() in asn.lower() or asname_score > 0.8 or provider_score > 0.8:
                count += 1
                print('"{}": "{}",'.format(asn, provider))
        if count == 0:
            print('NONE FOUND: "{}" ("{}")'.format(asname,provider))

In [70]:
display(Markdown('### Top Mailserver Nameservers'))
df_with_ns['mailserver_registered_domain_nameserver1'].value_counts().to_frame()[:30]

### Top Mailserver Nameservers

,mailserver_registered_domain_nameserver1
ns1.google.com,5270713
a1-245.akam.net,638790
ns1.msft.net,540042
ns-1and1.ui-dns.biz,295656
edns4.ultradns.biz,264138
dns10.ovh.net,256981
ns1.clev1.net,197145
dns1.p03.nsone.net,132351
a.b-one-dns.net,119823
ns1.p256.dynect.net,94131


In [71]:
display(Markdown('### Quick summary reports for analysis ...'))
#mask = df_with_ns.mailserver_registered_domain_nameserver1.fillna('').str.contains('cscdns.net')
#mask = df_with_ns.maxmind_asname == 'Linode, LLC'
#mask = df_with_ns.maxmind_asname == 'DIGITALOCEAN-ASN'
#mask = df_with_ns.maxmind_asname == 'RACKSPACE'
#mask = df_with_ns.maxmind_asname.fillna('').str.contains('Alibaba')
#mask = df_with_ns.mailserver_registered_domain == 'h-email.net'
#mask = df_with_ns.mx_ip_ptr_registered_domain == 'deteque.com'
#mask = df_with_ns.mx_ip_ptr_registered_domain == 'spamcloud.md'
#mask = df_with_ns.domain.isin(f100)
mask = df_with_ns.mailserver_registered_domain == 'psmtp.com'

display(df_with_ns[mask].mailserver_registered_domain.value_counts().to_frame())
display(df_with_ns[mask].maxmind_asname.value_counts().to_frame())
display(df_with_ns[mask].mx_ip_ptr_registered_domain.value_counts().to_frame())
display(df_with_ns[mask].mailserver_registered_domain_nameserver1.value_counts().to_frame())
display(df_with_ns[mask].email_provider.value_counts().to_frame())
display(df_with_ns[mask].mailserver.value_counts().to_frame())
#display(df_with_ns[mask].domain.value_counts().to_frame())

### Quick summary reports for analysis ...

,mailserver_registered_domain
psmtp.com,6241


,maxmind_asname


,mx_ip_ptr_registered_domain
,6241


,mailserver_registered_domain_nameserver1
ns1.google.com,6241


,email_provider
Postini,6241


,mailserver
quinstreet.com.mail7.psmtp.com,149
quinstreet.com.mail8.psmtp.com,149
quinstreet.com.mail6.psmtp.com,149
quinstreet.com.mail5.psmtp.com,149
prestoinc.com.s9a1.psmtp.com,21
...,...
everythingtrackandfield.com.s7a1.psmtp.com,1
nepsport.hu.s201a1.psmtp.com,1
jsi-service.com.mail10.psmtp.com,1
copperinfo.com.s5b2.psmtp.com,1


In [72]:
end_time = datetime.datetime.now()
print(start_time)
print(end_time)

2020-06-23 09:21:50.359162
2020-06-23 10:53:30.162473
